In [1]:
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime
import pickle

In [2]:
path = 'IP_Project_DataBase'

In [3]:
os.listdir(path)

['2110990025.jpg',
 '2110990152.jpg',
 '2110990153.jpg',
 '2110990155.jpg',
 '2110990157.jpg',
 '2110990161.jpg',
 '2110990163.jpg',
 '2110990165.jpg',
 '2110990184.jpg',
 '2110990186.jpg',
 '2110990188.jpg',
 '2110990190.jpg',
 '2110990375_Bhuvan.jpg']

In [4]:
images = []
classNames = []
mylist = os.listdir(path)
for cl in mylist:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

In [5]:
classNames

['2110990025',
 '2110990152',
 '2110990153',
 '2110990155',
 '2110990157',
 '2110990161',
 '2110990163',
 '2110990165',
 '2110990184',
 '2110990186',
 '2110990188',
 '2110990190',
 '2110990375_Bhuvan']

In [6]:
def findEncodings(images):
    img_count = 0
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)[0]
        encodeList.append(encoded_face)
        img_count += 1
        print(img_count)
    return encodeList
encoded_face_train = findEncodings(images)

In [7]:
with open('processed_images.pickle', 'wb') as f:
    pickle.dump(encoded_face_train, f)

In [15]:
with open('processed_images.pickle', 'rb') as f:
    encoded_face_train = pickle.load(f)

In [22]:
def markAttendance(name):
    print("doing attendance")
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            time = now.strftime('%I:%M:%S:%p')
            date = now.strftime('%d-%B-%Y')
            f.writelines(f'{name}, {time}, {date} \n')

In [21]:
# take pictures from webcam 
attendance_arr = []
count_frame = 0
count_flag = 0
cap  = cv2.VideoCapture(0)
try :
    while True:
        success, img = cap.read()
        imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
        faces_in_frame = face_recognition.face_locations(imgS)
        encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)
        for encode_face, faceloc in zip(encoded_faces,faces_in_frame):
            matches = face_recognition.compare_faces(encoded_face_train, encode_face)
            faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
            matchIndex = np.argmin(faceDist)
            if matches[matchIndex]:
                name = classNames[matchIndex].upper().lower()
                if min(faceDist) > .5:
                    name = "unknown"
                y1,x2,y2,x1 = faceloc
                # since we scaled down by 4 times
                y1, x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                cv2.rectangle(img, (x1,y2-35),(x2,y2), (0,255,0), cv2.FILLED)
                if count_flag <= 0 :
                    count_flag = 1
                    flag = False
#                 cv2.putText(img,name, (x1+6,y2-5), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                attendance_arr.append(name)
                if len(attendance_arr) == 10:
                    maxi = 0
                    res = attendance_arr[0]
                    for i in attendance_arr:
                        freq = attendance_arr.count(i)
                        if freq > maxi:
                            maxi = freq
                            res = i
                    attendance_arr.clear()
                    if (res != "unknown") :
                        markAttendance(res)
                        flag = True
                        count_flag = 10
#                     print(res)
                if flag :
                    if name != "unknown" :
                        cv2.putText(img,f"Attendance Done of {name}", (x1-200,y2-5), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                else :
                    if name == "unknown" :
                        cv2.putText(img,name, (x1+6,y2-5), cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
                    else :
                        cv2.putText(img,name, (x1+6,y2-5), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
        count_frame += 1
        count_flag -= 1
#         print(count_frame)
        cv2.imshow('webcam', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    print("Something went wrong")
cap.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

doing attendance
doing attendance


In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("Attendance.csv")

In [23]:
df.tail()

,Roll Number,Time,Date
0,2110990165,06:23:29:PM,20-March-2024
1,2110990163,06:24:51:PM,20-March-2024
2,2110990165,03:02:40:PM,23-March-2024
3,2110990165,03:02:44:PM,23-March-2024
4,2110990165,03:02:49:PM,23-March-2024
